In [1]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [4]:
from google.colab import drive
drive.mount('/content/drive')

FILE = open('/content/drive/MyDrive/Colab Notebooks/NLP_labs/sentences.txt')
SENTENCES = FILE.read().split('. ')[:-1]

max_tokens = 10000
tokens_count = 0

for sent in SENTENCES:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(SENTENCES))
avg_tokens
SENTENCES

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['which input is most helpful is classifying data without error? These input-weight products are summed and then the sum is passed through a node’s so-called activation function, to determine whether and to what extent that signal should progress further through the network to affect the ultimate outcome, say, an act of classification',
 'And thanks to the demand from gamers around the world, GPUs (graphics processing units) make it possible for us to leverage deep learning algorithms to build and train models with impressive results in a time-efficient manner! So to all the parents who don’t like your kids playing games: Gaming has its silver lining…',
 'Is the dataset you need publicly available, or can you create it (with a data annotation service like Scale or AWS Mechanical Turk)? In this example, spam emails would be labeled as spam, and the labels would enable the algorithm to map from inputs to the classifications you care about',
 '(Neural networks can also extract features th

In [5]:
targets = []

for sent in SENTENCES:
  targets.append({'Sentence':sent, 'Target':'Deep learning'})
  
targets = pd.DataFrame(data=targets, columns=['Sentence', 'Target'])

targets['Label'] = targets['Target']
targets.replace({'Label':{'Deep learning':0, 'Neural network':1}}, inplace=True)
targets

,Sentence,Target,Label
0,which input is most helpful is classifying dat...,Deep learning,0
1,And thanks to the demand from gamers around th...,Deep learning,0
2,"Is the dataset you need publicly available, or...",Deep learning,0
3,(Neural networks can also extract features tha...,Deep learning,0
4,Deep learning algorithms have improved over th...,Deep learning,0
...,...,...,...
94,That’s why deep learning is important,Deep learning,0
95,Deep learning maps inputs to outputs,Deep learning,0
96,This is known as supervised learning,Deep learning,0
97,Learn to build AI in simulations »,Deep learning,0


#### Tokenization&Embedding

In [7]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(targets['Sentence'])

text_vectorizer(targets['Sentence'])

<tf.Tensor: shape=(99, 18), dtype=int64, numpy=
array([[ 26,  36,   6, ...,   5,  69,   2],
       [  5, 302,   3, ...,  16, 168,  18],
       [  6,   2, 123, ..., 659, 347,  34],
       ...,
       [ 43,   6, 193, ...,   0,   0,   0],
       [ 58,   3, 127, ...,   0,   0,   0],
       [167, 664, 368, ...,   0,   0,   0]])>

In [8]:
embedding = layers.Embedding(input_dim=max_tokens, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=avg_tokens) 

embedding(text_vectorizer(targets['Sentence']))

<tf.Tensor: shape=(99, 18, 128), dtype=float32, numpy=
array([[[ 0.0163821 , -0.03467585,  0.04596889, ..., -0.04864758,
         -0.01153984, -0.0036635 ],
        [ 0.02479143,  0.04932738,  0.00519253, ...,  0.01194167,
         -0.00252515,  0.04962761],
        [-0.02088852, -0.01808979, -0.01267672, ..., -0.01657037,
         -0.02902266,  0.01460754],
        ...,
        [ 0.03170617, -0.02810094,  0.04729507, ...,  0.04991596,
         -0.01322588, -0.02670757],
        [-0.03329312,  0.02579292,  0.00674375, ..., -0.00065199,
          0.00714965,  0.00541302],
        [ 0.01508147,  0.00076116,  0.00783252, ..., -0.00686599,
         -0.02813537,  0.01126839]],

       [[ 0.03170617, -0.02810094,  0.04729507, ...,  0.04991596,
         -0.01322588, -0.02670757],
        [ 0.02002202,  0.00723224,  0.037806  , ...,  0.04593069,
         -0.01134557, -0.00291819],
        [-0.00132812, -0.02557141, -0.0497974 , ..., -0.00237709,
          0.03247285,  0.00704537],
        ...,

#### Train/Test Split

In [15]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(targets, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
96,This is known as supervised learning,Deep learning,0
64,"Also, this “oil refinery” is improving on both...",Deep learning,0
13,There could be a lot of “fossil fuels” hidden ...,Deep learning,0
26,"Neural networks are a set of algorithms, model...",Deep learning,0
41,Anomaly detection: The flipside of detecting s...,Deep learning,0
...,...,...,...
76,"Today, just about doing anything online will g...",Deep learning,0
62,One can say that the matrix of weights governs...,Deep learning,0
10,Other types of problems include anomaly detect...,Deep learning,0
55,You want to get some results and provide infor...,Deep learning,0


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: USE та USE-FT, здійснити порівняння моделей

In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_NB = Pipeline([
                    ("tfidf", TfidfVectorizer()), 
                    ("clf", MultinomialNB()) 
])

# Fit the pipeline to the training data
model_NB.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_NB.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 100.00%


### Sentence Encoder

In [17]:
import tensorflow_hub as hub
url="https://tfhub.dev/google/universal-sentence-encoder/4"

sentence_encoder_layer = hub.KerasLayer(url,input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

model_use = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_use.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_use.summary()


Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


### Sentence Encoder with fine tuning

In [19]:
tf_hub_embedding_layer = hub.KerasLayer(url,input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=True, 
                                        name="USE-FT")

model_use_ft = tf.keras.Sequential([
  tf_hub_embedding_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_use_ft")

# Compile model
model_use_ft.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


model_use_ft.summary()


Model: "model_use_ft"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE-FT (KerasLayer)         (None, 512)               256797824 
                                                                 
 dense_4 (Dense)             (None, 64)                32832     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 256,830,721
Non-trainable params: 0
_________________________________________________________________


In [20]:
model_use_ft_history=model_use_ft.fit(train_split['Sentence'],train_split['Label'],epochs=10,
                        validation_data=(test_split['Sentence'],test_split['Label']))


Epoch 1/10
3/3 [==============================] - 10s 550ms/step - loss: 0.6553 - accuracy: 0.6203 - val_loss: 0.4793 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 1s 254ms/step - loss: 0.4345 - accuracy: 1.0000 - val_loss: 0.3233 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 1s 263ms/step - loss: 0.2942 - accuracy: 1.0000 - val_loss: 0.2322 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 1s 259ms/step - loss: 0.2142 - accuracy: 1.0000 - val_loss: 0.1761 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 1s 252ms/step - loss: 0.1634 - accuracy: 1.0000 - val_loss: 0.1360 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 1s 249ms/step - loss: 0.1264 - accuracy: 1.0000 - val_loss: 0.1051 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 1s 252ms/step - loss: 0.0978 - accuracy: 1.0000 - val_loss: 0.0812 - val_accuracy: 1.0000
Epoch 8/10
3/3 [===

## Здійснити порівняння ефективності моделей

### NB

In [21]:
calculate_results(test_split['Label'], model_NB.predict(test_split['Sentence']))

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

### USE

In [22]:
model_use_preds=model_use.predict(test_split['Sentence'])
model_use_preds=tf.squeeze(tf.round(model_use_preds))
calculate_results(test_split['Label'],model_use_preds)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 75.0, 'f1': 0.8571428571428571, 'precision': 1.0, 'recall': 0.75}

### USE-FT

In [23]:
model_use_ft_preds=model_use_ft.predict(test_split['Sentence'])
model_use_ft_preds=tf.squeeze(tf.round(model_use_ft_preds))
calculate_results(test_split['Label'],model_use_ft_preds)

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}